<div class="well well-sm">
    
**Note**: This notebook mimics the Seurat Clustering tutorial up to the point where UMAP is called
    
    https://satijalab.org/seurat/v3.1/pbmc3k_tutorial.html
</div>

# To do:
- set `%%R -w 800 -h 450` as the default

In [ ]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.display(genepattern.session.register("https://cloud.genepattern.org/gp", "", ""))

In [ ]:
import os 
import urllib.request
import subprocess
import rpy2
%load_ext nbtools.r_support

# for setting default plot size see
# https://stackoverflow.com/questions/40745163/jupyter-notebook-rpy2-rmagics-how-to-set-the-default-plot-size
# but its prettybrittle - fails if you call it twice or change after the function is set




@genepattern.build_ui(name="Notebook Set up", description="Setup the R and Python environments for the rest of this notebook. Downloads the example dataset to the notebook server.", 
                      parameters={
                            "output_var": {
                                "hide": True,
                            }
})
def notebook_setup():
    %load_ext rpy2.ipython
    import seaborn as sns
    sns.set(rc={'figure.figsize':(8,4.5)})
    os.makedirs('data/pbmc3k/', exist_ok=True)
    url = 'https://s3-us-west-2.amazonaws.com/10x.files/samples/cell/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz'
    urllib.request.urlretrieve(url, 'data/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz')

    subprocess.run(["tar", "-xvf", "data/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz",
                "--directory", "data/pbmc3k/" ])
    
    
    print("Done.")


<div id="header" class="fluid-row" style="color: #333333; font-family: 'open sans' , 'arial' , sans-serif; font-size: 14px;">
<h1 id="Seurat---Guided-Clustering-Tutorial" class="title toc-ignore" style="margin-top: 20px; margin-bottom: 10px; font-size: 38px; padding: 0px; font-family: 'ubuntu' , 'tahoma' , 'helvetica neue' , 'helvetica' , 'arial' , sans-serif;" data-toc-modified-id="Seurat---Guided-Clustering-Tutorial-2"><a id="Seurat---Guided-Clustering-Tutorial-2" class="toc-mod-link"></a>Seurat - Guided Clustering Tutorial</h1>
<h4 id="Compiled:-October-08,-2019" class="date" style="margin: 10px 0px; font-size: 18px; padding: 0px; font-family: 'ubuntu' , 'tahoma' , 'helvetica neue' , 'helvetica' , 'arial' , sans-serif;" data-toc-modified-id="Compiled:-October-08,-2019-2.0.0.1"><a id="Compiled:-October-08,-2019-2.0.0.1" class="toc-mod-link"></a>Compiled: October 08, 2019</h4>
</div>
<hr style="overflow: visible; margin: 20px 0px; padding: 0px; color: #333333; font-family: 'open sans' , 'arial' , sans-serif; font-size: 14px;" />
<div id="setup-the-seurat-object" class="section level3" style="color: #333333; font-family: 'open sans' , 'arial' , sans-serif; font-size: 14px;">
<h3 id="Set-up-the-Seurat-Object" style="margin: 20px 0px 10px; font-size: 26px; padding: 0px; font-family: 'ubuntu' , 'tahoma' , 'helvetica neue' , 'helvetica' , 'arial' , sans-serif;" data-toc-modified-id="Set-up-the-Seurat-Object-2.0.1"><a id="Set-up-the-Seurat-Object-2.0.1" class="toc-mod-link"></a>Set up the Seurat Object</h3>
</div>

<p style="margin-bottom: 10px; padding: 0px; color: #373d3f; font-size: 16px; text-align: justify;">For this tutorial, we will be analyzing the a dataset of <span style="text-decoration: underline;"><strong>Peripheral Blood Mononuclear Cells (PBMC)</strong></span> freely available from 10X Genomics. There are 2,700 single cells that were sequenced on the Illumina NextSeq 500. The raw data was downloaded as part of the "Notebook Set up" cell (above) and the original files can be found&nbsp;<a style="color: #dd4814;" href="https://s3-us-west-2.amazonaws.com/10x.files/samples/cell/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz" target="_blank" rel="noopener">here</a>.</p>

<p style="margin-bottom: 10px; padding: 0px; color: #373d3f; font-size: 16px; text-align: justify;">
The three files for this dataset include
<ul style="margin-bottom: 10px; color: #373d3f; font-size: 16px; text-align: justify;"><li>matrix.mtx - triples of gene ID index, cell barcode index, UMI count. </li>
    <li>barcodes.tsv - the barcodes referenced by the indices in the MTX file.</li>
    <li>genes.tsv - all the annotated genes, one per row. Referenced by the indices in the MTX file.</li>
</ul>
</p>
<p style="margin-bottom: 10px; padding: 0px; color: #373d3f; font-size: 16px; text-align: justify;">
Details of the file formats can be found on the <a href="https://kb.10xgenomics.com/hc/en-us/articles/115000794686-How-is-the-MEX-format-used-for-the-gene-barcode-matrices-">10X Genomics knowledge base</a>.  You can run the following cell to see the first few lines of each of these files.
</p>

In [ ]:
from itertools import islice


@genepattern.build_ui( name="Show File Heads", description="Shows the first N lines of the 3 10X Genomics PBMC data files", 
                      parameters={
                            "N": {"default": "5"},
                            "output_var": {
                                "hide": True,
                            }
})
def showHeadOfFiles(N):
    print("File:  data/pbmc3k/filtered_gene_bc_matrices/hg19/Matrix.mtx\n")
    with open("data/pbmc3k/filtered_gene_bc_matrices/hg19/matrix.mtx") as f:
        for line in islice(f, N):
            print("\t"+line)
    print("\nFile:  data/pbmc3k/filtered_gene_bc_matrices/hg19/barcodes.tsv\n")
    with open("data/pbmc3k/filtered_gene_bc_matrices/hg19/barcodes.tsv") as f:
        for line in islice(f, N):
            print("\t"+line)
    print("\nFile: data/pbmc3k/filtered_gene_bc_matrices/hg19/genes.tsv\n")
    with open("data/pbmc3k/filtered_gene_bc_matrices/hg19/genes.tsv") as f:
        for line in islice(f, N):
            print("\t" + line)
        

<p style="margin-bottom: 10px; padding: 0px; color: #373d3f; font-size: 16px; text-align: justify;">Fundamentally, Seurat starts with a feature (e.g. gene) expression matrix. The expected format of the input matrix is features x cells. Here we use the information embedded in the three files described above for this purpose.</p>

<p style="margin-bottom: 10px; padding: 0px; color: #373d3f; font-size: 16px; text-align: justify;">We start by reading in the data. The&nbsp;<code style="font-size: 14.4px; color: inherit; border: 1px solid #9ba4a7; border-radius: 4px; background-color: rgba(0 , 0 , 0 , 0.04); white-space: pre;">Read10X</code>&nbsp;function reads in the output of the&nbsp;<a style="color: #dd4814;" href="https://support.10xgenomics.com/single-cell-gene-expression/software/pipelines/latest/what-is-cell-ranger" target="_blank" rel="noopener">cellranger</a>&nbsp;pipeline from 10X, returning a unique molecular identified (UMI) count matrix. The values in this matrix represent the number of molecules for each feature (i.e.&nbsp;gene; row) that are detected in each cell (column).</p>

<p style="margin-bottom: 10px; padding: 0px; color: #373d3f; font-size: 16px; text-align: justify;">We next use the count matrix to create a&nbsp;<code style="font-size: 14.4px; color: inherit; border: 1px solid #9ba4a7; border-radius: 4px; background-color: rgba(0 , 0 , 0 , 0.04); white-space: pre;">Seurat</code>&nbsp;object. The object serves as a container that includes both data (i.e., the count matrix) and analysis (e.g., principal component analysis -- PCA, or clustering results) for a single-cell dataset. For a technical discussion of the&nbsp;<code style="font-size: 14.4px; color: inherit; border: 1px solid #9ba4a7; border-radius: 4px; background-color: rgba(0 , 0 , 0 , 0.04); white-space: pre;">Seurat</code>&nbsp;object structure, check out Seurat's&nbsp;<a style="color: #dd4814;" href="https://github.com/satijalab/seurat/wiki" target="_blank" rel="noopener">GitHub Wiki</a>. For example, the count matrix is stored in&nbsp;<code style="font-size: 14.4px; color: inherit; border: 1px solid #9ba4a7; border-radius: 4px; background-color: rgba(0 , 0 , 0 , 0.04); white-space: pre;">pbmc[["RNA"]]@counts</code>.</p>

In [ ]:
%%r_build_ui { "name": "Setup Seurat Objects", "parameters": { "output_var": { "hide": "True" } } }

setupR <- function(){
    print('Loading libraries')
#     library(dplyr)
    library(Seurat)
    library(scater)
    fig_height=450
    fig_width=800
    # Load the PBMC dataset
    print('Loading the dataset.')
    pbmc.data <- Read10X(data.dir = "data/pbmc3k/filtered_gene_bc_matrices/hg19/")
    
#     raw_counts <- readSparseCounts(file="https://datasets.genepattern.org/data/module_support_files/Conos/HNSCC_noribo.txt")
#     hnscc <- CreateSeuratObject(counts = raw_counts, project = "HNSCC")
    
    # Initialize the Seurat object with the raw (non-normalized data).
    pbmc <- CreateSeuratObject(counts = pbmc.data, project = "pbmc3k", min.cells = 3, min.features = 200)
    print('Done')
#     return(hnscc)
    return(pbmc)
}
pbmc <- setupR()

<h3 style="margin: 20px 0px 10px; font-size: 26px; padding: 0px; font-family: Ubuntu, Tahoma, 'Helvetica Neue', Helvetica, Arial, sans-serif; color: #333333;">Standard pre-processing workflow</h3>
<p style="margin-bottom: 10px; padding: 0px; color: #373d3f; font-size: 16px; text-align: justify; font-family: 'Open Sans', arial, sans-serif;">The steps below encompass the standard pre-processing workflow for scRNA-seq data in Seurat. These represent the selection and filtration of cells based on QC metrics, data normalization and scaling, and the detection of highly variable features.</p>
<div id="qc-and-selecting-cells-for-further-analysis" class="section level4" style="color: #333333; font-family: 'Open Sans', arial, sans-serif; font-size: 14px;">
<h4 style="margin: 10px 0px; font-size: 18px; padding: 0px; font-family: Ubuntu, Tahoma, 'Helvetica Neue', Helvetica, Arial, sans-serif;">QC and selecting cells for further analysis</h4>
<p style="margin-bottom: 10px; padding: 0px; color: #373d3f; font-size: 16px; text-align: justify;">Seurat allows you to easily explore QC metrics and filter cells based on any user-defined criteria. A few QC metrics&nbsp;<a style="color: #dd4814; text-decoration-line: underline;" href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4758103/">commonly used</a>&nbsp;by the community include</p>
<ul style="margin-right: 0px; margin-bottom: 10px; margin-left: 35px; padding: 0px; font-size: 16px;">
<li style="margin-bottom: 5px; text-align: justify;">The number of unique genes detected in each cell.
<ul style="margin-right: 0px; margin-left: 35px; padding: 0px 0px 0px 15px;">
<li style="margin-bottom: 5px; text-align: justify;">Low-quality cells or empty droplets will often have very few genes</li>
<li style="margin-bottom: 5px; text-align: justify;">Cell doublets or multiplets may exhibit an aberrantly high gene count</li>
</ul>
</li>
<li style="margin-bottom: 5px; text-align: justify;">Similarly, the total number of molecules detected within a cell (correlates strongly with unique genes)</li>
<li style="margin-bottom: 5px; text-align: justify;">The percentage of reads that map to the mitochondrial genome
<ul style="margin-right: 0px; margin-left: 35px; padding: 0px 0px 0px 15px;">
<li style="margin-bottom: 5px; text-align: justify;">Low-quality / dying cells often exhibit extensive mitochondrial contamination</li>
<li style="margin-bottom: 5px; text-align: justify;">We calculate mitochondrial QC metrics with the&nbsp;<code style="font-size: 14.4px; color: inherit; word-break: break-word; border: 1px solid #9ba4a7; border-radius: 4px; background-color: rgba(0, 0, 0, 0.04); white-space: pre;">PercentageFeatureSet</code>&nbsp;function, which calculates the percentage of counts originating from a set of features</li>
<li style="margin-bottom: 5px; text-align: justify;">We use the set of all genes starting with&nbsp;<code style="font-size: 14.4px; color: inherit; word-break: break-word; border: 1px solid #9ba4a7; border-radius: 4px; background-color: rgba(0, 0, 0, 0.04); white-space: pre;">MT-</code>&nbsp;as a set of mitochondrial genes</li>
</ul>
</li>
</ul>
</div>

In [ ]:
%%r_build_ui { "name": "Add Mitochondrial QC Metrics", "parameters": { "column_name": { "type": "string", "default":"percent.mt" },"pattern": { "type": "string", "default":"MT-" }, "output_var": { "hide": "True" } } }

set_mito_qc <- function(colName, pat) {
    
    pattern <- paste("^", trimws(pat, which = "both"), sep="")
    
    # The [[ operator can add columns to object metadata. This is a great place to stash QC stats
    pbmc[[colName]] <- PercentageFeatureSet(pbmc, pattern = pattern)
    print("done!")
    return(pbmc)
}


pbmc <- set_mito_qc(column_name, pattern)

<p style="margin-bottom: 10px; padding: 0px; color: #373d3f; font-size: 16px; text-align: justify; font-family: 'Open Sans', arial, sans-serif;">In the example below, we visualize QC metrics, and use these to filter cells.</p>
<ul style="margin-right: 0px; margin-bottom: 10px; margin-left: 35px; padding: 0px; font-size: 16px; color: #333333; font-family: 'Open Sans', arial, sans-serif;">
<li style="margin-bottom: 5px; text-align: justify;">We filter cells that have unique feature counts over 2,500 or less than 200</li>
<li style="margin-bottom: 5px; text-align: justify;">We filter cells that have &gt;5% mitochondrial counts</li>
</ul>
</p>

In [ ]:
%%r_build_ui { "width": 10, "height": 300, "name": "Triple Violin Plot", "parameters": { "first_feature": { "type": "string", "default":"nFeature_RNA" }, "second_feature":{ "type": "string", "default":"nCount_RNA"}, "third_feature": { "type": "string", "default":"percent.mt" }, "output_var":{"hide":"True"} } }
# Visualize QC metrics as a violin plot
#VlnPlot(pbmc, features = c(first_feature, second_feature, third_feature), ncol = 3)
tripleViolin <- function(first, second, third){
     
    feats <- c(first, second, third)
    plot(VlnPlot(pbmc, features = feats, ncol = 3, combine=TRUE), fig.height=5, fig.width=15)
    return("")
}

tripleViolin(first_feature, second_feature, third_feature)

In [ ]:
%%r_build_ui { "name": "Double Scatter Plot", "parameters": { "x_axis": { "type": "String", "default":"nCount_RNA" }, "y1": { "type": "string", "default":"percent.mt"}, "y2": {"type":"string", "default":"nFeature_RNA" }, "output_var": {"hide": "True"}  } }
# %%R -w 800 -h 450
# FeatureScatter is typically used to visualize feature-feature relationships, but can be used
# for anything calculated by the object, i.e. columns in object metadata, PC scores etc.
Feature_Scatter_Plot <- function(first_x, first_y, second_y){
    plot1 <- FeatureScatter(pbmc, feature1 = first_x, feature2 = first_y)
    plot2 <- FeatureScatter(pbmc, feature1 = first_x, feature2 = second_y)
    plot(CombinePlots(plots = list(plot1, plot2)))
    return("")
}
Feature_Scatter_Plot(x_axis,y1,y2)

<p style="margin-bottom: 10px; padding: 0px; color: #373d3f; font-size: 16px; text-align: justify; font-family: 'Open Sans', arial, sans-serif;">
<b>Filtering data</b>
<br><br>
Here we select which cells and genes we will filter out of our dataset based on the plots above.
</p>

In [ ]:
%%R
print(pbmc)

In [ ]:
%%r_build_ui { "name": "Subset Data", "parameters": { "min_n_features": { "type": "number", "default":"200" },"max_n_features": { "type": "number", "default":"2500" },"max_percent_mitochondrial": { "type": "number", "default":"5" }, "output_var": { "hide": "True" } } }

my_subset <- function(min_n_features, max_n_features, max_percent_mitochondrial){
#     print(pbmc)
    pbmc <- subset(pbmc, subset = nFeature_RNA > min_n_features & nFeature_RNA < max_n_features & percent.mt < max_percent_mitochondrial)
#     print(pbmc)
    print('filtering done!')
    return(pbmc)
}

pbmc <- my_subset(min_n_features, max_n_features, max_percent_mitochonrial)

<h3 style="margin: 20px 0px 10px; font-size: 26px; padding: 0px; font-family: Ubuntu, Tahoma, 'Helvetica Neue', Helvetica, Arial, sans-serif; color: #333333;">Normalizing the data</h3>
<p style="margin-bottom: 10px; padding: 0px; color: #373d3f; font-size: 16px; text-align: justify; font-family: 'Open Sans', arial, sans-serif;">After removing unwanted cells from the dataset, the next step is to normalize the data. By default, we employ a global-scaling normalization method &ldquo;LogNormalize&rdquo; that normalizes the feature expression measurements for each cell by the total expression, multiplies this by a scale factor (10,000 by default), and log-transforms the result. Normalized values are stored in&nbsp;<code style="font-size: 14.4px; color: inherit; word-break: break-word; border: 1px solid #9ba4a7; border-radius: 4px; background-color: rgba(0, 0, 0, 0.04); white-space: pre;">pbmc[["RNA"]]@data</code>. The methods we will use in this notebook all assume log-normalized data, other methods may not make that assumption and other normalization methods can be considered.</p>

In [ ]:
%%r_build_ui { "name": "Normalize", "parameters": { "method": { "type": "string", "default":"LogNormalize" },"scale_factor": { "type": "number", "default":"10000" }, "output_var": { "hide": "True" } } }

norm_pbmc <- function(meth, scale){
    pbmc <- NormalizeData(pbmc, normalization.method = meth, scale.factor = scale)
    print('Normalization done!')
    return(pbmc)
}

pbmc <- norm_pbmc(method, scale_factor)

<h3 style="margin: 20px 0px 10px; font-size: 26px; padding: 0px; font-family: Ubuntu, Tahoma, 'Helvetica Neue', Helvetica, Arial, sans-serif; color: #333333;">Identification of highly variable features (feature selection)</h3>
<p style="margin-bottom: 10px; padding: 0px; color: #373d3f; font-size: 16px; text-align: justify; font-family: 'Open Sans', arial, sans-serif;">We next calculate a subset of features that exhibit high cell-to-cell variation in the dataset (i.e, they are highly expressed in some cells, and lowly expressed in others).<a style="color: #dd4814; text-decoration-line: underline;" href="https://www.nature.com/articles/nmeth.2645">The literature</a>&nbsp;suggests that focusing on these genes in downstream analysis helps to highlight biological signal in single-cell datasets.</p>
<p style="margin-bottom: 10px; padding: 0px; color: #373d3f; font-size: 16px; text-align: justify; font-family: 'Open Sans', arial, sans-serif;">Seurat3's  Seurat3 is described in detail&nbsp;<a style="color: #dd4814; text-decoration-line: underline;" href="https://www.biorxiv.org/content/early/2018/11/02/460147.full.pdf">here</a>, and improves on previous versions by directly modeling the mean-variance relationship inherent in single-cell data, and is implemented in the&nbsp;<code style="font-size: 14.4px; color: inherit; word-break: break-word; border: 1px solid #9ba4a7; border-radius: 4px; background-color: rgba(0, 0, 0, 0.04); white-space: pre;">FindVariableFeatures</code>&nbsp;function. By default, ot returns 2,000 features per dataset. These will be used in downstream analysis, like PCA.</p>

In [ ]:
%%r_build_ui { "name": "Feature Selection", "parameters": { "method": { "type": "string", "default":"vst","hide":"True" },"num_features": { "type": "number", "default":"2000" }, "num_to_label":{"type": "number", "default": "10", "description": "label the top N features in the plot."}, "output_var": { "hide": "True" } } }
#%%R -w 800 -h 450

feat_sel_plot <- function(meth, nFeat, nLabel){
    pbmc <- FindVariableFeatures(pbmc, selection.method = meth, nfeatures = nFeat)

    # Identify the 10 most highly variable genes
    top10 <- head(VariableFeatures(pbmc), nLabel)

    # plot variable features with and without labels
    plot1 <- VariableFeaturePlot(pbmc)
    plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)
    print(plot2)
    #plot(CombinePlots(plots = list(plot1, plot2)))
    return(pbmc)
}

pbmc <- feat_sel_plot(method, num_features, num_to_label)

<h3 style="margin: 20px 0px 10px; font-size: 26px; padding: 0px; font-family: Ubuntu, Tahoma, 'Helvetica Neue', Helvetica, Arial, sans-serif; color: #333333;">Scaling the data</h3>
<p style="margin-bottom: 10px; padding: 0px; color: #373d3f; font-size: 16px; text-align: justify; font-family: 'Open Sans', arial, sans-serif;">Next, we apply a linear transformation (&lsquo;scaling&rsquo;) that is a standard pre-processing step prior to dimensional reduction techniques like PCA. The&nbsp;<code style="font-size: 14.4px; color: inherit; word-break: break-word; border: 1px solid #9ba4a7; border-radius: 4px; background-color: rgba(0, 0, 0, 0.04); white-space: pre;">ScaleData</code>&nbsp;function:</p>
<ul style="margin-right: 0px; margin-bottom: 10px; margin-left: 35px; padding: 0px; font-size: 16px; color: #333333; font-family: 'Open Sans', arial, sans-serif;">
<li style="margin-bottom: 5px; text-align: justify;">Shifts the expression of each gene, so that the mean expression across cells is 0</li>
<li style="margin-bottom: 5px; text-align: justify;">Scales the expression of each gene, so that the variance across cells is 1
<ul style="margin-right: 0px; margin-left: 35px; padding: 0px 0px 0px 15px;">
<li style="margin-bottom: 5px; text-align: justify;">This step gives equal weight in downstream analyses (which may apply weights based on variance), so that highly-expressed genes do not dominate</li>
</ul>
</li>
<li style="margin-bottom: 5px; text-align: justify;">The results of this are stored in&nbsp;<code style="font-size: 14.4px; color: inherit; word-break: break-word; border: 1px solid #9ba4a7; border-radius: 4px; background-color: rgba(0, 0, 0, 0.04); white-space: pre;">pbmc[["RNA"]]@scale.data</code></li>
</ul>

In [ ]:
%%r_build_ui {"name": "Scale Data", "parameters": {"output_var":{"hide": "True"}}}
myscale <- function(pbmc){
    all.genes <- rownames(pbmc)
    pbmc <- ScaleData(pbmc, features = all.genes)
    print('done!')
    return(pbmc)
}
pbmc <- myscale(pbmc)

<h3 style="margin: 20px 0px 10px; font-size: 26px; padding: 0px; font-family: Ubuntu, Tahoma, 'Helvetica Neue', Helvetica, Arial, sans-serif; color: #333333;">Perform Principal Component Analysis (PCA)</h3>
<p style="margin-bottom: 10px; padding: 0px; color: #373d3f; font-size: 16px; text-align: justify; font-family: 'Open Sans', arial, sans-serif;">Next we perform PCA on the scaled data. By default, only the previously determined variable features are used as input, but can be defined using&nbsp;<code style="font-size: 14.4px; color: inherit; word-break: break-word; border: 1px solid #9ba4a7; border-radius: 4px; background-color: rgba(0, 0, 0, 0.04); white-space: pre;">features</code>&nbsp;argument if you wish to choose a different subset.</p>

In [ ]:
%%r_build_ui {"name":"Perform PCA", "parameters":{"output_var":{"hide":"True"}}}

mypca <-function(pbmc){ 
    feats <- VariableFeatures(object = pbmc)
    pbmc <-RunPCA(pbmc, features = feats, nfeatures.print=5)
   
    return(pbmc)
}
pbmc <- mypca(pbmc)

<p><span style="color: #373d3f; font-family: 'Open Sans', arial, sans-serif; font-size: 16px; text-align: justify;">Seurat provides several useful ways of visualizing both cells and features that define the PCA, including&nbsp;</span><code style="font-size: 14.4px; color: #373d3f; word-break: break-word; border: 1px solid #9ba4a7; border-radius: 4px; background-color: rgba(0, 0, 0, 0.04); white-space: pre; text-align: justify;">VizDimReduction</code><span style="color: #373d3f; font-family: 'Open Sans', arial, sans-serif; font-size: 16px; text-align: justify;">,&nbsp;</span><code style="font-size: 14.4px; color: #373d3f; word-break: break-word; border: 1px solid #9ba4a7; border-radius: 4px; background-color: rgba(0, 0, 0, 0.04); white-space: pre; text-align: justify;">DimPlot</code><span style="color: #373d3f; font-family: 'Open Sans', arial, sans-serif; font-size: 16px; text-align: justify;">, and&nbsp;</span><code style="font-size: 14.4px; color: #373d3f; word-break: break-word; border: 1px solid #9ba4a7; border-radius: 4px; background-color: rgba(0, 0, 0, 0.04); white-space: pre; text-align: justify;">DimHeatmap</code></p>

In [ ]:
%%r_build_ui {"name":"Examine Features", "parameters": {"num_dims": {"type":"number", "default":"5"}, "num_features":{"type":"number", "default":"5"}, "output_var": {"hide": "True"} }}

ex_feats <- function(dim, nFeat){
    n_dims = 1:dim
    print(n_dims)
    print(pbmc[["pca"]], dims = n_dims, nfeatures = nFeat)
    return("")
}
ex_feats(num_dims, num_features)

In [ ]:
%%r_build_ui {"name":"Vizualize Dimension Loadings", "parameters": {"num_dims": {"type":"string", "default":"2"}, "output_var": {"hide": "True"} }}

vdl <- function(nDims){
    dim_range = c(1,strtoi(nDims))
    print(VizDimLoadings(pbmc, dims = dim_range, reduction = "pca"))
    return("")
}
vdl(num_dims)

In [ ]:
%%r_build_ui {"name":"Vizualize Dimension Plot", "parameters": { "output_var": {"hide": "True"} }}

vdp <- function(p1){
    
    plot(DimPlot(pbmc, reduction = "pca"))
    return("")
}
vdp()

<p><span style="color: #373d3f; font-family: 'Open Sans', arial, sans-serif; font-size: 16px; text-align: justify;">In particular&nbsp;</span><code style="font-size: 14.4px; color: #373d3f; word-break: break-word; border: 1px solid #9ba4a7; border-radius: 4px; background-color: rgba(0, 0, 0, 0.04); white-space: pre; text-align: justify;">DimHeatmap</code><span style="color: #373d3f; font-family: 'Open Sans', arial, sans-serif; font-size: 16px; text-align: justify;">&nbsp;allows for easy exploration of the primary sources of heterogeneity in a dataset, and can be useful when trying to decide how many PCs to include for further downstream analyses. Both cells and features are ordered according to their PCA scores. Setting&nbsp;</span><code style="font-size: 14.4px; color: #373d3f; word-break: break-word; border: 1px solid #9ba4a7; border-radius: 4px; background-color: rgba(0, 0, 0, 0.04); white-space: pre; text-align: justify;">cells</code><span style="color: #373d3f; font-family: 'Open Sans', arial, sans-serif; font-size: 16px; text-align: justify;">&nbsp;to a number plots the &lsquo;extreme&rsquo; cells on both ends of the spectrum, which dramatically speeds plotting for large datasets. Though clearly a supervised analysis, we find this to be a valuable tool for exploring correlated feature sets.</span></p>

In [ ]:
%%r_build_ui {"name":"DimHeatMap", "parameters": { "num_dims":{"type":"string", "default": "1"}, "cells":{"type": "number","default":"500"}, "output_var": {"hide": "True"} }}

vdhm <- function(nd,c){    
    if (nd == 1){
        dim_range = 1
    } else {
        dim_range = c(1:strtoi(nd))
    }
    
    print(DimHeatmap(pbmc, dims = dim_range, cells = c, balanced = TRUE))
    return(pbmc)
}
vdhm(num_dims, cells)

In [ ]:
%%r_build_ui {"name":"DimHeatMap", "parameters": { "num_dims":{"type":"string", "default": "15"}, "cells":{"type": "number","default":"500"}, "output_var": {"hide": "True"} }}

vdhm(num_dims, cells)

<h3 id="Determine-the-&lsquo;dimensionality&rsquo;-of-the-dataset" style="margin: 20px 0px 10px; font-size: 26px; padding: 0px; font-family: 'ubuntu' , 'tahoma' , 'helvetica neue' , 'helvetica' , 'arial' , sans-serif; color: #333333;" data-toc-modified-id="Determine-the-&lsquo;dimensionality&rsquo;-of-the-dataset-2.1.5"><a id="Determine-the-&lsquo;dimensionality&rsquo;-of-the-dataset-2.1.5" class="toc-mod-link"></a>Determine the &lsquo;dimensionality&rsquo; of the dataset</h3>
<p style="margin-bottom: 10px; padding: 0px; color: #373d3f; font-size: 16px; text-align: justify; font-family: 'open sans' , 'arial' , sans-serif;">To overcome the extensive technical noise in any single feature for scRNA-seq data, Seurat clusters cells based on their PCA scores, with each PC essentially representing a &lsquo;metafeature&rsquo; that combines information across a correlated feature set. The top principal components therefore represent a robust compression of the dataset. However, how many componenets should we choose to include? 10? 20? 100?</p>

<p><span style="color: #373d3f; font-family: 'open sans' , 'arial' , sans-serif; font-size: 16px; text-align: justify;">To do this, we will use a heuristic method which generates an &lsquo;Elbow plot&rsquo;: a ranking of principal components based on the percentage of variance explained by each one (</span><code style="font-size: 14.4px; color: #373d3f; border: 1px solid #9ba4a7; border-radius: 4px; background-color: rgba(0 , 0 , 0 , 0.04); white-space: pre; text-align: justify;">ElbowPlot</code><span style="color: #373d3f; font-family: 'open sans' , 'arial' , sans-serif; font-size: 16px; text-align: justify;">&nbsp;function). In this example, we can observe an &lsquo;elbow&rsquo; around PC9-10, suggesting that the majority of true signal is captured in the first 10 PCs.</span></p>

In [ ]:
%%r_build_ui {"name":"Elbow Plot", "parameters": {  "output_var": {"hide": "True"} }}
ebp <- function(){
    plot(ElbowPlot(pbmc))
    return(pbmc)
}
ebp()

<p style="margin-bottom: 10px; padding: 0px; color: #373d3f; font-size: 16px; text-align: justify; font-family: 'open sans' , 'arial' , sans-serif;">Identifying the true dimensionality of a dataset &ndash; can be challenging/uncertain for the user. Here we have shone a quick method but users are advised to read on other ones [INSERT LINKS]</p>

<p><span style="color: #e03e2d;">INSERT LINKS ABOVE</span></p>

<div class="well well-sm">
Export to RDATA file ("save")
</div>

In [ ]:
%%r_build_ui {"name":"Save preprocessed dataset", "parameters": {  "file_name": {"default":"pbmc_preprocessed.rds"}, "output_var": {"hide": "True"} }}
save_it <- function(fileName){
    saveRDS(pbmc, file = fileName)
    print("Saved file!")
    return(pbmc)
}
save_it(file_name)

In [ ]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.display(genepattern.session.register("https://beta.genepattern.org/gp", "", ""))

In [ ]:
seuratclustering_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00408')
seuratclustering_job_spec = seuratclustering_task.make_job_spec()
seuratclustering_job_spec.set_parameter("input.seurat.rds.file", "")
seuratclustering_job_spec.set_parameter("output.filename", "<input.seurat.rds.file_basename>.clustered")
seuratclustering_job_spec.set_parameter("maximum_dimension", "10")
seuratclustering_job_spec.set_parameter("resolution", "0.5")
seuratclustering_job_spec.set_parameter("reduction", "umap")
seuratclustering_job_spec.set_parameter("job.memory", "2 Gb")
seuratclustering_job_spec.set_parameter("job.queue", "gp-new-beta")
seuratclustering_job_spec.set_parameter("job.cpuCount", "1")
seuratclustering_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(seuratclustering_task)


job55930 = gp.GPJob(genepattern.session.get(0), 55930)
genepattern.display(job55930)

# Read in the rds file from the module, then show cluster markers as in the Seurat tutorial 

https://satijalab.org/seurat/v3.1/pbmc3k_tutorial.html

[I can't get the UMAP to look the same as the tutorial...]

Module input:  
https://datasets.genepattern.org/data/module_support_files/SeuratClustering/pbmc_preprocessed.rds

Module output:  
https://datasets.genepattern.org/data/module_support_files/SeuratClustering/postmarker.rds